In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout, LSTM, GRU, Bidirectional, Embedding

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.io as pio
tokenizer=tf.keras.preprocessing.text.Tokenizer()
from keras_preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Layer, Dense, Dropout, MultiHeadAttention, LayerNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
from IPython.display import set_matplotlib_formats 
plt.style.use('ggplot')
sns.set_style('whitegrid')
sns.set(font_scale=1.5)
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")
import gc

# Pallets used for visualizations
color= "Spectral"
color_plt = ListedColormap(sns.color_palette(color).as_hex())
color_hist = 'teal'
two_colors = [ sns.color_palette(color)[0], sns.color_palette(color)[5]]
three_colors = [ sns.color_palette(color)[5],sns.color_palette(color)[2], sns.color_palette(color)[0]]

In [ ]:
#Read fastq file #57 colon #46 gastric #49 normal colon
def readFastq(filename):
    """Reads FASTQ file and remove the special characters!"""
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            qual = fh.readline().rstrip() #base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [ ]:
#with tf.device('CPU:0'):
seq1, quals = readFastq('/kaggle/input/coloncancer/SRR22422957_1.fastq')

In [ ]:
df = pd.DataFrame(seq1)
#df = df.loc[:10000000]
df = df.rename(columns = {0: 'sequences'})
df['label'] = 'ColonCancer'

In [ ]:
#Downsampling
downsampling_rate = 0.38
num_rows = int(downsampling_rate * df.shape[0])
downsampled_df = df.sample(n=num_rows)

In [ ]:
#with tf.device('CPU:0'):
seq2, quals = readFastq('/kaggle/input/throidcancer/SRR22570931_1.fastq')
#with tf.device('CPU:0'):
df1 = pd.DataFrame(seq2)
#df1 = df1.loc[:200000]
df1 = df1.rename(columns = {0: 'sequences'})
df1['label'] = 'ThyroidCancer'

In [ ]:
#with tf.device('CPU:0'):
seq3, quals = readFastq('/kaggle/input/urothelial-carcinoma/SRR20665001_1.fastq')
#with tf.device('GPU:0'):
df2 = pd.DataFrame(seq3)
#df2 = df2.loc[:200000]
df2 = df2.rename(columns = {0: 'sequences'})
df2['label'] = 'UrithelialCarcinoma'

In [ ]:
df_new = pd.concat([downsampled_df, df1, df2], ignore_index = True)

In [ ]:
sns.countplot(x = 'label', data = df_new)

In [ ]:
df_uni = df_new.drop_duplicates(subset = ['sequences'], keep = 'first')

In [ ]:
# Select the rows corresponding to the third class
class3_rows = df_new[df_new['label'] == 'ThyroidCancer']

# Randomly select a specified number of rows from the third class
downsampled_class3 = class3_rows.sample(n=500000)

# Combine the downsampled third class with the rest of the DataFrame
df_new = pd.concat([df_new[df_new['label'] != 'ThroidCancer'], downsampled_class3])
df_new

In [ ]:
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size=4):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [ ]:
df_uni["sequences"]=df_uni.apply(lambda x: getKmers(x["sequences"]),axis=1)
df_uni.head()

In [ ]:
df_texts=list(df_uni["sequences"])
for item in range(len(df_texts)):
    df_texts[item] = ' '.join(df_texts[item])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_uni['sequences'], df_uni['label'], test_size = .2)

In [ ]:
#pipeline
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
pipe_lr = Pipeline(steps = [('cv', CountVectorizer()),('lr', LogisticRegression())])

In [ ]:
#training
pipe_lr.fit(x_train, y_train)

In [ ]:
pipe_lr.score(x_test, y_test)

In [ ]:
y_pred = pipe_lr.predict(x_test)

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt


y_probas = pipe_lr.predict_proba(x_test)
skplt.metrics.plot_roc_curve(y_test, y_probas)
plt.show()

In [ ]:
skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize=True)
plt.show()

In [ ]:
#callibration
from sklearn.calibration import CalibratedClassifierCV
# Use CalibratedClassifierCV to train a calibrated version of the model
calibrated_model = CalibratedClassifierCV(pipe_lr, cv=5, method="sigmoid")
calibrated_model.fit(x_train, y_train)
calibrated_model.score(x_test, y_test)

In [ ]:
y_probas = calibrated_model.predict_proba(x_test)
skplt.metrics.plot_roc_curve(y_test, y_probas)
plt.show()